# JupyterHubのセットアップ

---

VCノードにJupyterHub環境を構築します。

## 概要

このNotebookで構築するJupyterHubの構成要素を以下に示します。

![<構成図表示エラー>](images/arch.png)

> このNotebookで構築するJupyterHubではMoodleとの連携を前提としています。

### 事前に準備が必要となるものについて

このNotebookを実行するにあたって、あらかじめ準備が必要となるものについて以下に記します。

* VCノード構築
* JupyterHubのサーバ証明書
    - JupyterHubではHTTPSによる公開を行うためサーバ証明書とその秘密鍵を準備する必要があります。
* 機関のLDAPサーバ
* Moodleシステム
    - JHコンテナからMoodleで使用しているDBにアクセス可能であること。（受講生一覧を取得するため）

### UnitGroup名

JupyterHubの構築環境となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にして、UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'JupyterHub'

ugroup_name = ''

### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となる各VCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

In [ ]:
target_nodes = f'{ugroup_name}_worker'

!ansible {target_nodes} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

## パラメータの設定

### 各種設定項目

JupyterHubシステムに関する各種設定項目を指定します。

ここで設定された項目はansible変数として保存され、ファイルのmanagerノードへの配置時にdocker-compose.ymlとjupyterhub_config.iniに反映されます。

#### 必須設定項目

In [ ]:
# JupyterHubのホスト名(FQDN) (例) 'vcc1045.vcp-handson.org'
master_fqdn = ''

# moodleDB接続情報
# DB名 (例) 'moodle'
moodle_db_name = ''
# DBユーザ (例) 'moodle'
moodle_db_user = ''
# DBホスト (例) '172.30.2.110'
moodle_db_host = ''
# DBパスワード (例) 'password'
moodle_db_password = ''

# グローバルLDAP接続情報
# ホスト (例) '172.30.2.120:1389'
global_ldap_server = '' # ポートはデフォルト(389)であれば、省略可能
# ベースDN (例) 'ou=People,dc=ldap,dc=server,dc=sample,dc=jp'
global_ldap_base_dn = ''
# パスワード (例) 'PassWordDesu'
global_ldap_password = ''


In [ ]:
# LTI用 ※デフォルト指定無し
# コンシューマーキー (例) 'cz5j8p2v3c0zjvpad6vyt4xqtkkvyn4d4faba18g2rll0odlerk524k3ipczbzqh'
lti_consumer_key = ''
# 共有シークレット (例) 'cz5j8p2v3c0zjvpad6vyt4xqtkkvyn4d4faba18g2rll0odlerk524k3ipczbzqh'
lti_secret = ''

In [ ]:
# LTI用 ※上のセルで指定が無い場合に生成するためのセル
import random, string

# ランダムな値を生成する場合
n = 64
if not 'lti_consumer_key' in locals() or not lti_consumer_key:
    lti_consumer_key = ''.join(random.choices(string.ascii_lowercase + string.digits, k=n))
if not 'lti_secret' in locals() or not lti_secret:
    lti_secret = ''.join(random.choices(string.ascii_lowercase + string.digits, k=n))

#### 任意設定項目

In [ ]:
# JupyterhubDB接続情報
jh_db_name = 'jupyterthub'
jh_db_user = 'jupyter'
jh_db_password = 'PassWordDesu'
jh_db_host = 'mariadb:3306'

# Jupyterhub 初期設定
jupyterhub_admin_users = ['admin', 'user01']

# single-user notebookコンテナ リソース制限
teacher_mem_limit = '1024M'
student_mem_limit = '512M'
mem_guarantee = '256M'
cpu_limit = '0.5'
cpu_guarantee = '0.2'

# notebookイメージ
singleuser_image = 'swarm_yamaguchi:1.7'

# dockerネットワーク
swarm_network = 'swarm_jupyterhub-net'

# メールアドレスドメイン設定
email_domain = 'server.sampl.jp'

### オーバーレイネットワークの指定

JupyterhubHubのコンテナはDocker Swarmのオーバーレイネットワークで実行されます。オーバーレイネットワークに割り当てるサブネットを指定します。

> 基本的にはデフォルト値からの変更は不要ですが、VCノードに割り当てられているIPアドレスと範囲が重複している場合は他のサブネットに変更して下さい。

In [ ]:
# (例)
# jupyterhub_backend = '10.1.0.0/20'

jupyterhub_backend = '10.1.0.0/20'

指定されたサブネットがVCノードに割り当てられているIPアドレスと重なっていないことをチェックします。次のセルを実行してエラーとならないことを確認してください。

In [ ]:
!ansible-playbook -v -e jupyterhub_backend={jupyterhub_backend} -l {ugroup_name} \
    playbooks/check-subnet.yml

### パラメータの保存

ここまで指定したパラメータを ansible の変数として `group_vars`ファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

gvars.update({
    'master_fqdn': master_fqdn,
    'jh_db_name': jh_db_name,
    'jh_db_user': jh_db_user,
    'jh_db_password': jh_db_password,
    'jh_db_host': jh_db_host,
    'moodle_db_name': moodle_db_name,
    'moodle_db_user': moodle_db_user,
    'moodle_db_host': moodle_db_host,
    'moodle_db_password': moodle_db_password,
    'jupyterhub_backend': jupyterhub_backend,
    'global_ldap_server': global_ldap_server,
    'global_ldap_base_dn': global_ldap_base_dn,
    'global_ldap_password': global_ldap_password,
    'email_domain': email_domain,
    'lti_consumer_key': lti_consumer_key,
    'lti_secret': lti_secret,
    'jupyterhub_admin_users': jupyterhub_admin_users,
    'teacher_mem_limit': teacher_mem_limit,
    'student_mem_limit': student_mem_limit,
    'mem_guarantee': mem_guarantee,
    'singleuser_image': singleuser_image,
    'singleuser_image_no_tag': singleuser_image.split(':')[0],
    'swarm_network': swarm_network,
    'cpu_limit': cpu_limit,
    'cpu_guarantee': cpu_guarantee,
})

with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)

## JupyterHubのセットアップ

JupyterwareHubの構成要素となる、各コンテナのセットアップを行います。

### Nginx

Nginx コンテナに関するセットアップを行います。

#### 証明書の配置

Nginxコンテナで使用するサーバ証明書の配置を確認します。

サーバ証明書を配置するディレクトリを作成します。

In [ ]:
!ansible {target_hub} -b -m file -a \
    'path={{{{base_dir}}}} state=directory owner={{{{ansible_user}}}}'
!ansible {target_hub} -b -m file -a \
    'path={{{{base_dir}}}}/certs state=directory owner={{{{ansible_user}}}}'

作成したディレクトリ`/srv/jupyterhub/certs`に証明書、秘密鍵を配置してください。出どころなどの情報を必要以上に残さないためにNotebookからの操作ではなく、ターミナルなどから **managerノードに ssh でログインして操作を行ってください**。

配置する証明書などのファイル名は以下のようにしてください。

* サーバ証明書と中間CA証明書を連結したもの
    - `/srv/jupyterhub/certs/fullchain.pem`
* 秘密鍵
    - `/srv/jupyterhub/certs/privkey.pem`
    - パスフレーズを無しにする

sshでログインする manager ノードのIPアドレスを確認します。表示されたIPアドレスに対して、ユーザ名`vcp`と「VCノード作成」のNotebookで設定したSSHの秘密鍵を指定することで manager ノードにsshでログインできます。

In [ ]:
!ansible {target_hub} -c local -m debug -a 'var=vc_ipaddress'

#### 証明書の配置確認

証明書の配置後に以下のセルを実行し、正しく配置できていることを確認します。

秘密鍵の内容を表示してみます。

In [ ]:
!ansible {target_hub} -a \
    'openssl rsa -noout -text -in  {{{{certs_dir}}}}/privkey.pem'

中間CA証明書を連結したサーバ証明書の内容を表示してみます。

In [ ]:
!ansible {target_hub} -a \
    'openssl x509 -noout -text -in {{{{certs_dir}}}}/fullchain.pem'

証明書の owner, group などの値を利用環境に合わせて以下のように設定します。

* owner: 33
* group: 33

この設定はコンテナ内では以下のように設定したことに相当します。

* owner: `www-data`
* group: `www-data`

In [ ]:
cert_owner = 33
cert_group = 33
!ansible {target_hub} -b -m file -a \
    'path={{{{certs_dir}}}} owner={cert_owner} \
    group={cert_group} state=directory'
!ansible {target_hub} -b -m file -a \
    'path={{{{certs_dir}}}}/fullchain.pem \
    owner={cert_owner} group={cert_group}'
!ansible {target_hub} -b -m file -a \
    'path={{{{certs_dir}}}}/privkey.pem \
    owner={cert_owner} group={cert_group} mode=0600'

### JupyterHub

JupyterHubコンテナに関するセットアップを行います。

#### JupyterHubコンテナに関するファイルを準備する

JupyterHubコンテナを実行するために必要となるファイルを準備する Ansible Playbook を実行します。

Playbook では以下の処理を行います。

* コンテナイメージのビルド

コンテナイメージのビルドに必要なファイルを配置します。

まず、ファイルを格納するディレクトリを作成

In [ ]:
# 先にディレクトリ作成
!ansible {target_hub} -b -m file -a \
    'path=/jupytershare/nbgrader state=directory owner={{{{ansible_user}}}}'
!ansible {target_hub} -b -m file -a \
    'path=/jupytershare/class state=directory owner={{{{ansible_user}}}}'

ファイルの配置をチェックモードで試行

In [ ]:
# /jupytershare に必要なファイル配置
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=template/directories/jupytershare/nbgrader dest=/jupytershare'
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=template/directories/jupytershare/class dest=/jupytershare'

# /home に必要なファイル配置
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=template/directories/skelton dest=/home'

# jupyterhubシステム構築に必要なファイル配置
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=template/jupyterhub dest={{{{base_dir}}}}'
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=template/notebook dest={{{{base_dir}}}}'

実際にファイルを配置

In [ ]:
# /jupytershare に必要なファイル配置
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/directories/jupytershare/nbgrader dest=/jupytershare'
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/directories/jupytershare/class dest=/jupytershare'

# /home に必要なファイル配置
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/directories/skelton dest=/home'

# jupyterhubシステム構築に必要なファイル配置
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/jupyterhub dest={{{{base_dir}}}}'
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/notebook dest={{{{base_dir}}}}'


#### Jupyterhubイメージのビルド

実際にビルドする前にドライラン（チェックモード）でAnsibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/setup-jupyterhub.yml || true

実際にコンテナイメージのビルドを行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/setup-jupyterhub.yml

JupyterHubのコンテナイメージが存在することを確認します。

In [ ]:
!ansible {target_hub} -a \
    'docker images jh-simple'

JupyterHubコンテナのために配置したファイルを確認します。

In [ ]:
!ansible {target_hub} -a 'tree {{{{jupyterhub_dir}}}}'

### single-user Jupyter Notebook server

single-user notebookコンテナイメージが各VCノードに存在するかを確認します。

イメージが存在しない場合、結果が`FAILED`になります。（その場合でも、ここではイメージの存否確認が出来れば良いので、セルの実行結果としては正常終了とします。）

In [ ]:
!ansible {ugroup_name} -m shell -a 'docker images | \
    grep -e "{{{{singleuser_image}}}}"' || true

#### single-user notebookコンテナイメージが存在しない場合

公開リポジトリからsingle-user notebookコンテナイメージをダウンロードします。

デフォルトのイメージであればビルドも可能です。

以下のセルを実行し、single-user notebookコンテナイメージをダウンロードしてください。

ダウンロードが成功した場合、以降のセルは実行せず、4章までスキップしてください。

In [ ]:
!ansible {ugroup_name} -a 'docker pull {{{{singleuser_image}}}}' || true

#### single-user notebookコンテナイメージをビルドする場合

`template/jh_notebook` にsingle-user notebookコンテナイメージをビルドするためのファイルを格納しています。

オリジナルのイメージを作成する場合は、この内容を置き換えてください。

workerノードに必要なファイルを格納するためのディレクトリを作成する。

In [ ]:
!ansible {ugroup_name} -b -m file -a \
    'path={{{{notebook_dir}}}} state=directory owner={{{{ansible_user}}}}'

notebookコンテナイメージビルドに必要なファイルを配置

まず、チェックモードで実行

In [ ]:
!ansible {ugroup_name} -CDv -m synchronize \
    -a 'src=template/notebook dest={{{{notebook_dir}}}}'

実際に配置

In [ ]:
!ansible {ugroup_name} -Dv -m synchronize \
    -a 'src=template/notebook dest={{{{notebook_dir}}}}'

#### single-user notebookコンテナイメージのビルド

実際のビルド前に、チェックモードで確認します。

In [ ]:
!ansible-playbook -l {ugroup_name} -CDv playbooks/setup-jupyter-notebook.yml || true

実際にビルドします。

In [ ]:
!ansible-playbook -l {ugroup_name} playbooks/setup-jupyter-notebook.yml

各VCノードのコンテナイメージ一覧を確認します。

In [ ]:
!ansible {ugroup_name} -m shell -a 'docker images | \
    grep -e "{{{{singleuser_image_no_tag}}}}"'

## コンテナの起動

### docker-compose.yml の配置

複数のコンテナに関する設定をまとめて扱うために `docker-compose.yml` を利用します。

ここまでに指定されたパラメータに応じた`docker-compose.yml`,`jupyterhub_config.ini`,`default.conf`を生成し、構築環境に配置します。

まずは、チェックモードで確認を行います。

In [ ]:
!ansible {target_hub} -CDv -m template \
    -a 'src=template/jupyterhub/docker-compose.yml dest={{{{base_dir}}}}/jupyterhub backup=yes'
!ansible {target_hub} -CDv -m template \
    -a 'src=template/jupyterhub/jupyterhub/jupyterhub_config.ini dest={{{{jupyterhub_dir}}}} backup=yes'
!ansible {target_hub} -CDv -m template \
    -a 'src=template/jupyterhub/nginx/default.conf dest={{{{base_dir}}}}/jupyterhub/nginx backup=yes'

実際に各ファイルの配置を行います。

In [ ]:
!ansible {target_hub} -Dv -m template \
    -a 'src=template/jupyterhub/docker-compose.yml dest={{{{base_dir}}}}/jupyterhub backup=yes'
!ansible {target_hub} -Dv -m template \
    -a 'src=template/jupyterhub/jupyterhub/jupyterhub_config.ini dest={{{{jupyterhub_dir}}}} backup=yes'
!ansible {target_hub} -Dv -m template \
    -a 'src=template/jupyterhub/nginx/default.conf dest={{{{base_dir}}}}/jupyterhub/nginx backup=yes'

### コンテナの起動

コンテナを起動します。

In [ ]:
!ansible {target_hub} -a 'chdir={{{{base_dir}}}}/jupyterhub \
    docker stack deploy -c docker-compose.yml {{{{ugroup_name}}}}'

コンテナの起動状態を確認します。

In [ ]:
!ansible {target_hub} -a 'docker stack ps {{{{ugroup_name}}}}'

`MariaDB` コンテナのログを表示してみます。

In [ ]:
!ansible {target_hub} -a 'docker service logs {{{{ugroup_name}}}}_mariadb'

`jupyterhub` コンテナのログを表示してみます。

> PostgreSQLに接続できないなどのエラーが表示されていないことを確認してください。

In [ ]:
!ansible {target_hub} -a 'docker service logs {{{{ugroup_name}}}}_hub'

`Nginx` コンテナのログを表示してみます。

> 証明書設定に誤りがあるなどのエラーが表示されていないことを確認してください。

In [ ]:
!ansible {target_hub} -a 'docker service logs {{{{ugroup_name}}}}_proxy'

## Moodleの外部ツールを設定する

moodleの外部ツール設定画面にて、Jupyterhubを外部ツールとして登録します。

次のセルを実行すると、設定に必要な情報を表示します。

それぞれ、下に表示される図を参考に設定してください。

In [ ]:
print(f'ツールURL: https://{master_fqdn}/hub/lti/launch')
print(f'コンシューマーキー: {lti_consumer_key}')
print(f'共有シークレット: {lti_secret}')

![<図表示エラー>](images/moodle_tools.png)